!pip install minisom

In [ ]:
import fitsio
from astropy.io import fits
import numpy as np
from minisom import MiniSom
import matplotlib as mpl
import healpy as hp
import matplotlib.pyplot as plt
from tqdm import tqdm

mpl.rcParams['figure.dpi'] = 200

In [ ]:
import sys
sys.path.append('../sources')
from cat2som import *
from OR_weights import *
from plot_som import *

# Prepare the data 

In [ ]:
Download the file KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits from

http://cuillin.roe.ac.uk/~cech/KiDS/KiDS-1000/K1000-SHEAR-9bandPZ-CATALOGUES/KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits

by running the following cell (please check the username and password on Lensing KiDS Wiki):

!wget -P ../../data --user= --password= http://cuillin.roe.ac.uk/~cech/KiDS/KiDS-1000/K1000-SHEAR-9bandPZ-CATALOGUES/KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits

Add this file to the data directory.

Now we creat a smaller subsample of the catalog to make the work faster:

kids_goldsample = fits.open('../../data/KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits')

data_table = kids_goldsample[1].data

rows_numbers_to_keep = np.random.choice(len(data_table), 1000000, replace=False)

fits.writeto('/net/home/fohlen13/yanza21/DATA/KiDS_data/KiDS_gold_smallsample.fits', kids_goldsample[1].data[rows_numbers_to_keep], overwrite=True)

In [ ]:
kids_goldsample = fitsio.FITS('../../data/KiDS_data/KiDS_gold_smallsample.fits')

# Data preprocessing

In [ ]:
columns_selected = ['MU_THRESHOLD', 'PSF_Q12', 'PSF_e1','PSF_e2', 'MAG_LIM_r', 'EXTINCTION_r']  # columns to be read from the catalog
columns_syst = ['MU_THRESHOLD', 'PSF_Q12', 'PSF_ell', 'MAG_LIM_r', 'EXTINCTION_r']  # columns to be used for the training (can be different from columns_selected because some columns may be mixed up)
column_edges = np.array([[20,30], [-2, 2], [-1,1],  [20, 30], [-1,1]])  # prior boundaries of each column, used for excluding outliers

training_data = []

for key in columns_selected:
    training_data.append(kids_goldsample[1][key][:10000].tolist())
training_data = np.array(training_data).T

radec = np.array(kids_goldsample[1]['ALPHA_J2000', 'DELTA_J2000'][:10000].tolist())  # read RA and Dec

e = (training_data.T[2]**2 + training_data.T[3]**2) ** 0.5
q = (1-e)/(e+1)  # calculate ellipticity from shear components
training_data = np.vstack([training_data.T[0], training_data.T[1], 1-q, training_data.T[4], training_data.T[5]]).T

### excluding outliers ###

outliers = np.zeros(training_data.shape[0])
for i in range(training_data.shape[1]):
    l = training_data.T[i]
        
    dist_to_mean = l - l.mean()
    std = np.std(l[~np.isnan(l)])
    
    outliers += np.logical_or(l<column_edges[i][0], l>column_edges[i][1])    
    outliers += np.logical_or(l==+np.inf, l==-np.inf)
    outliers += np.isnan(l)

radec_edges = np.array([[0,360], [-90, 90]])
radec = np.round(radec,3)

for i in range(2):
    radec_i = radec.T[i]
    outliers += np.isnan(radec_i)
    outliers += np.logical_or(radec_i<radec_edges[i][0], radec_i>radec_edges[i][1])    
    outliers += np.logical_or(radec_i==+np.inf, radec_i==-np.inf)
    
invalid_ind = np.where(outliers>0)[0]

radec = np.delete(radec, invalid_ind, axis=0)

training_data = np.delete(training_data, invalid_ind, axis=0)
training_data_raw = training_data.copy()
training_data = (training_data - np.mean(training_data, axis=0)) / np.std(training_data, axis=0)

# Define and train a SOM

In [ ]:
cat2som_obj = cat2som(training_data, columns_syst, sigma=1, learning_rate=1, 
              neighborhood_function='gaussian', topology='hexagonal', random_seed=0)

In [ ]:
cat2som_obj.train_som(5000)

# Test the SOM with a subset of the training data

In [ ]:
test_ind = np.arange(5000)
testing_data = training_data#[test_ind]
testing_radec = radec#[test_ind]
umatrix = cat2som_obj.get_activation_map(testing_data) 

In [ ]:
winner_x, winner_y, winner_1dind = cat2som_obj.get_winner_ind(testing_data)

## let's check MU_THRESHOLD of each SOM cell

In [ ]:
feature_map_dict = cat2som_obj.get_feature_map(testing_data, col_ind=0, x=winner_x.astype(np.int), y=winner_y.astype(np.int), activation_map=umatrix)
feature_map = feature_map_dict['feature_map']
feature_map_name = feature_map_dict['colname']

In [ ]:
import matplotlib.pyplot as plt

f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111)
ax.set_aspect('equal')

plot_som(ax, feature_map, topology='hex', colormap=cm.coolwarm, cbar_name=feature_map_name)

In [ ]:
# plot the number of sources in each SOM cell

f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111)

ax.set_aspect('equal')
plot_som(ax, umatrix, topology='hex', colormap=cm.coolwarm)

# Calculate and plot the organized random weights

In [ ]:
Ns = 256
weight_map = calculate_or_weights(Ns, testing_radec, winner_1dind)

In [ ]:
hp.cartview(weight_map, lonra=[125, 240], latra=[-5,4], xsize=8000, title='KiDS-North')

In [ ]:
hp.cartview(weight_map, lonra=[-31, 55], latra=[-37,-26], title='KiDS-South')


## Plot the spatial distribution of each systematics

In [ ]:
for sys_ind in range(training_data.shape[1]):
    #sys_ind = 1
    hp.cartview(np.ones_like(weight_map)*hp.UNSEEN, lonra=[-31, 55], latra=[-37,-26], title='KiDS-South, '+columns_syst[sys_ind], flip='geo', cbar=None)
    hp.projscatter(testing_radec.T[0], testing_radec.T[1], lonlat=True, c=testing_data.T[sys_ind], cmap='coolwarm', s=0.1, marker='s',
                   vmin=np.quantile(testing_data.T[sys_ind],0.01), vmax=np.quantile(testing_data.T[sys_ind], 0.99))

In [ ]:
for sys_ind in range(training_data.shape[1]):
    hp.cartview(np.ones_like(weight_map)*hp.UNSEEN, lonra=[125-180, 240-180], latra=[-5,4], title='KiDS-North, '+columns_syst[sys_ind], flip='geo', cbar=None)
    hp.projscatter(testing_radec.T[0]-180, testing_radec.T[1], lonlat=True, c=testing_data.T[sys_ind], cmap='coolwarm', s=.1, marker='s',
              vmin=np.quantile(testing_data.T[sys_ind],0.01), vmax=np.quantile(testing_data.T[sys_ind], 0.99))
